In [22]:
## Genera certificados considerando un archivo principal y otro de referencia. 
## El archivo de referencia contiene todos los valores de DNI_ID que deben recibir certificado


import pandas as pd
from docxtpl import DocxTemplate
import subprocess
import os

# Importar los archivos
folder_data = "/home/justus/Documentos/dev/nogit-data/"
#tabla_cluberos2025  = pd.read_csv("/home/justus/Documentos/dev/nogit-data/certificados_edicion_2025/cluberos_2025_rev_ok_250725_Consolidado.csv")
tabla_cluberos2025  = pd.read_excel(folder_data + "certificados_edicion_2025/cluberos_2025_rev_ok_250725_Consolidado.xlsx")
tabla_encuesta_cierre = pd.read_excel(folder_data + "certificados_edicion_2025/encuesta_cierre_cdecpe2025.xlsx")

## Recuperamos los DNI como string 
# Suponiendo que ya cargaste y limpiaste los DNI
tabla_cluberos2025["DNI_ID"] = tabla_cluberos2025["DNI_ID"].astype(str).str.replace(".0", "", regex=False)
tabla_cluberos2025["Application_ID"] = tabla_cluberos2025["Application_ID"].astype(str)
tabla_cluberos2025["User_ID"] = tabla_cluberos2025["User_ID"].astype(str)

tabla_encuesta_cierre["DNI_ID"] = tabla_encuesta_cierre["DNI_ID"].astype(str).str.replace(".0", "", regex=False)
tabla_encuesta_cierre["Teléfono"] = tabla_encuesta_cierre["Teléfono"].astype(str).str.replace(".0", "", regex=False)

## Esta sección agrega una nueva columna que son los que recibirán certificado a través del nombre completo
# Hacer merge solo con los DNI que existen en ambos (inner join)
df_combinado = pd.merge(
    tabla_cluberos2025,
    tabla_encuesta_cierre[["DNI_ID", "Nombre_certificado"]],  # aquí la columna que quieres jalar
    on="DNI_ID",
    how="outer"
)

## Finalmenten genera un excel con todos los que recibirán certificado 
# df_combinado ahora contiene todas las columnas de tabla_cluberos2025
# más la columna de tabla_encuesta_cierre que seleccionaste
df_combinado.to_excel("certificados_final_cluberos2025.xlsx", index=False)



In [23]:
## generación de certificados 

df_filtrado = df_combinado[df_combinado["Nombre_certificado"].notna()]
df_filtrado_sin_duplicados = df_filtrado.drop_duplicates(subset=["Nombre_certificado"], keep='last')

nombres = df_filtrado_sin_duplicados["Nombre_certificado"].tolist()
correos = df_filtrado_sin_duplicados["Email"].tolist()

list_email = []
for i, row_fil in df_filtrado_sin_duplicados.iterrows():
    doc = DocxTemplate("certificado_plantilla.docx")
    context = row_fil.to_dict()  # Diccionario con los valores de la fila
    doc.render(context)
    name = context["Nombre_certificado"]
    email = context["Email"]
    nombre = context["First_Name"]

    filename_docx = f"certificado_e2025_{name}_{email}.docx"
    filename_pdf = f"certificado_e2025_{name}_{email}.pdf"
    list_email.append([nombre, email])  # crea la lista de correos
    
    doc.save(filename_docx)

    # Convertir a PDF (si tienes LibreOffice instalado)
    subprocess.run(["libreoffice", "--headless", "--convert-to", "pdf", filename_docx])
    os.rename(filename_docx.replace(".docx", ".pdf"), filename_pdf)